In [28]:

import os
from openai import OpenAI
from langchain_core.runnables import Runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()


import os
print("Current key:", os.environ.get("HUGGINGFACE_API_KEY"))


Current key: None


In [79]:
import os
from openai import OpenAI
from langchain_core.runnables import Runnable
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class ChatModel2(Runnable):
    """
    Runnable wrapper around the OpenAI-compatible client (used for HuggingFace router).
    Works with LangChain Core pipelines (prompt | chatmodel2 | parser).
    """

    def __init__(self, model_name: str, api_key: str = None, base_url: str = None):
        super().__init__()
        # ✅ This line now looks for HUGGINGFACE_API_KEY, not HF_TOKEN
        key = api_key or os.environ.get("HF_TOKEN")
        if not key:
            raise ValueError("No Hugging Face API key found. Please set HUGGINGFACE_API_KEY in your .env file.")
        
        self.client = OpenAI(
            base_url = base_url or "https://router.huggingface.co/v1",
            api_key = key,
        )
        self.model_name = model_name

    def invoke(self, inputs, config=None, **kwargs):
        """Accepts dict, string, or ChatPromptValue"""
        try:
            prompt_text = inputs.to_string()
        except Exception:
            if isinstance(inputs, dict):
                prompt_text = inputs.get("prompt") or inputs.get("topic") or inputs.get("text")
            else:
                prompt_text = str(inputs)

        if not prompt_text or not prompt_text.strip():
            raise ValueError("No prompt text provided to ChatModel2.")

        completion = self.client.chat.completions.create(
            model = self.model_name,
            messages = [{"role": "user", "content": prompt_text}],
        )
        return completion.choices[0].message.content


In [80]:
chatmodel=ChatModel2("openai/gpt-oss-20b:groq")

chatmodel.invoke("what is the color of sky")

'The sky appears blue to our eyes during daylight because of Rayleigh scattering: shorter (blue) wavelengths of sunlight are scattered more strongly by the gases in the atmosphere. At sunrise or sunset, the scattering favors longer (red/orange) wavelengths, making the sky appear those colors.'

In [31]:
import os
from openai import OpenAI
from langchain_core.runnables import Runnable


class ChatModel2_120B(Runnable):
    """
    Runnable wrapper for the gpt-oss-120B model via the OpenAI-compatible Hugging Face router.
    """

    def __init__(self, model_name: str = "openai/gpt-oss-120b:groq",
                 api_key: str = None, base_url: str = None):
        super().__init__()
        self.client = OpenAI(
            base_url=base_url or "https://router.huggingface.co/v1",
            api_key=api_key or os.environ.get("HF_TOKEN"),
        )
        self.model_name = model_name

    def invoke(self, inputs=None, config=None, **kwargs):
        # Gracefully handle missing input
        if inputs is None:
            raise ValueError("The 'inputs' argument cannot be None.")

        # Normalize input
        if isinstance(inputs, dict):
            prompt_text = inputs.get("prompt") or inputs.get("topic") or str(inputs)
        else:
            prompt_text = str(inputs)

        prompt_text = prompt_text.strip()
        if not prompt_text:
            raise ValueError("No valid prompt text provided to ChatModel2_120B")

        # Generate completion
        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt_text}],
            **(config or {}),
            **kwargs,
        )

        # Return message content safely (some clients return dicts, others objects)
        choice = completion.choices[0]
        message = getattr(choice, "message", None)
        if isinstance(message, dict):
            return message.get("content", "")
        elif hasattr(message, "content"):
            return message.content
        else:
            return str(choice)


In [32]:
model=ChatModel2_120B()

In [33]:
print(chatmodel.invoke("what is 1+1"))

1 + 1 equals 2.


# RunnablePassthrough 

##### 💠 It does not do anything to input data
##### 💠 Lets see a eg so that we can get some understanding


In [47]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda

In [48]:
chain = RunnablePassthrough()

In [49]:

chain.invoke("Raghav")

'Raghav'

In [50]:
def russian_lastname(name:str) -> str:
    return f"{name} Sama"

In [51]:
chain=RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("Raghav")

'Raghav Sama'

# RunnableParallel

### 🛹We will use the runnnableparallel for running tasks in parallel
### 🛹This is probably the most important and most useful Runnabel from Langchain

In [52]:
from langchain_core.runnables import RunnableParallel


In [53]:
chain=RunnableParallel(
    {
        "operation_1":RunnablePassthrough(),
        "opearation2":RunnableLambda(russian_lastname)
    }
)

In [54]:
chain.invoke("Raghav")

{'operation_1': 'Raghav', 'opearation2': 'Raghav Sama'}

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [56]:
prompt=ChatPromptTemplate.from_template("tell me a curious fact about {player}")

output_parser=StrOutputParser()

In [57]:
def russian_lastname_from_dictonary(person):
    return person["name"] + "ovicH"

In [58]:
chain2=RunnableParallel(
    {
        "player":RunnablePassthrough(),
        "opearation2":RunnableLambda(russian_lastname_from_dictonary),
        "operation3":RunnablePassthrough(),

    }
) | prompt | chatmodel| output_parser


In [59]:
chain2.invoke({"name1": "Jordan","name":"Abhram"})

'Here’s a neat connection between the two names:\n\n- **Abraham** (often rendered as *Abram* in older translations) is the biblical patriarch who journeys from Ur to the land that God promised to his descendants.  \n- **Jordan** is the famous river that runs through the Middle East and is the waterway Abraham crossed when he entered the land of Canaan.\n\n**Curious fact:** In the Book of Genesis, Abraham and his family actually cross the Jordan River as part of the covenantal journey. This crossing is a pivotal moment—just as the name “Jordan” itself means “to descend,” Abraham’s name, meaning “father of many,” heralds the descent of a new, diverse peoples into the promised territory. It’s a little poetic that the patriarch who becomes the “father of many” literally steps into the river that bears a name meaning “to descend.”'

# Lets see a more use of 

In [60]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [81]:
import os
from huggingface_hub import InferenceClient

class QwenEmbeddingModel:
    def __init__(self, api_key=None, provider="nebius", model="Qwen/Qwen3-Embedding-8B"):
        self.api_key = api_key or os.getenv("HUGGINGFACE_API_KEY")
        if not self.api_key:
            raise ValueError("Please set HUGGINGFACE_API_KEY environment variable or pass api_key explicitly.")
        
        self.model = model
        self.client = InferenceClient(provider=provider, api_key=self.api_key)

    def embed_query(self, text: str):
        """Embed a single text and return a vector"""
        result = self.client.feature_extraction(text, model=self.model)
        return result[0]

    def embed_documents(self, texts: list[str]):
        """Embed multiple texts and return a list of vectors"""
        result = self.client.feature_extraction(texts, model=self.model)
        return result

    def __call__(self, text: str):
        """Allow the model to be called directly by FAISS or LangChain"""
        return self.embed_query(text)

In [82]:
import os
import numpy as np
from huggingface_hub import InferenceClient

class SentenceTransformerEmbeddingModel:
    def __init__(self, api_key=None, provider="hf-inference", model="sentence-transformers/all-MiniLM-L6-v2"):
        self.api_key = api_key or os.getenv("HF_TOKEN")
        if not self.api_key:
            raise ValueError("Please set HF_TOKEN environment variable or pass api_key explicitly.")
        
        self.model = model
        self.client = InferenceClient(provider=provider, api_key=self.api_key)

    def embed_query(self, text: str):
        result = self.client.feature_extraction(text, model=self.model)
        # Ensure it's 2D for FAISS (1 sample, N dims)
        return np.array(result).reshape(1, -1)

    def embed_documents(self, texts: list[str]):
        result = self.client.feature_extraction(texts, model=self.model)
        return np.array(result)

    def __call__(self, text: str):
        return self.embed_query(text)


In [83]:
loader=TextLoader(r'txt_files/ai_knowledge_base.txt',encoding="utf-8")
docs = loader.load()
text = " ".join([doc.page_content for doc in docs])


In [105]:
embedding_model=QwenEmbeddingModel(api_key=os.getenv("HF_TOKEN"))
embedding_model2=SentenceTransformerEmbeddingModel()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [106]:
text_split=splitter.split_text(text=text)

TypeError: expected string or bytes-like object

In [86]:
vector_store=FAISS.from_texts(texts=text_split,embedding=embedding_model2)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [87]:
retriver=vector_store.as_retriever()

In [88]:
template=""" Answer the question based only on the following context:
{context}

Question: {question}


"""

In [89]:
prompt=ChatPromptTemplate.from_template(template)

In [90]:
retrieval_chain=(RunnableParallel({"context":retriver,"question":RunnablePassthrough()}) | prompt | chatmodel | StrOutputParser()
                 )

In [94]:
answer=retrieval_chain.invoke("Compare convolutional neural networks (CNNs) and transformers in terms of architecture, data requirements, and computational cost. Why have transformers started replacing CNNs in computer vision tasks?")

ValueError: too many values to unpack (expected 2)

In [95]:
print(answer)

NameError: name 'answer' is not defined

In [96]:
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [97]:
newchatmodel=ChatModel2("openai/gpt-oss-20b:groq")

In [98]:
loader=TextLoader(r'txt_files/ai_knowledge_base.txt',encoding="utf-8")
docs = loader.load()
text = " ".join([doc.page_content for doc in docs])


In [99]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader(r"txt_files/ai_knowledge_base.txt", encoding="utf-8")
docs = loader.load()
text = [doc.page_content for doc in docs]
split_texts = splitter.split_text(" ".join(text))


In [100]:
vector_store2 = FAISS.from_texts(texts=split_texts, embedding=embedding_model2)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [101]:
retriver2=vector_store2.as_retriever()
template=""" Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following Language : {language}
"""
prompt=ChatPromptTemplate.from_template(template)

In [102]:
chain3 = (
    {
        "context": itemgetter("question")
        | retriver2
        | (lambda docs: "\n\n".join([d.page_content for d in docs])),
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | chatmodel
    | StrOutputParser()
)

In [103]:
chain3.invoke({"question": "Compare convolutional neural networks (CNNs) and transformers in terms of architecture, data requirements, and computational cost. Why have transformers started replacing CNNs in computer vision tasks? "})

ValueError: too many values to unpack (expected 2)